# Quaternions

## Representation
Quaternion represented by:
    q = a + bi + cj + dk = cos(t/2) + (u1, u2, u3)\*sin(t/2)
and the conjugate is
    q* = a - bi - cj - dk

## geodesic_distance
The **difference rotation quaternion** that represents the difference rotation is defined as **r≜pq∗**
The distance between rotations represented by unit quaternions p and q is the **angle of the difference rotation** represented by the unit quaternion **r=pq∗**.

So, from the Hamilton product of quaternions, we can derive that the r's scalar part is equal to:

    cos(t/2) = p1q1 + p2q2 + p3q3 + p4q4
So that

    t = 2 arcos ( | p1q1 + p2q2 + p3q3 + p4q4 | ) = 2 arcos ( |<p,q>| )
And this represent the distance between rotations or **geodesic distance**.

## quaternion_from_matrix
From https://d3cw3dd2w32x2b.cloudfront.net/wp-content/uploads/2015/01/matrix-to-quat.pdf but: 
* changing representation to adapt to us (i.e. move w (our a) from the last position to the first one).
* Altered to work with the column vector convention instead of row vectors

Code inspired by https://github.com/KieranWynn/pyquaternion/blob/master/pyquaternion/quaternion.py.



In [2]:
def quaternion_from_matrix(matrix):
    import numpy as np
    from math import sqrt
    """Initialise from matrix representation
    Create a Quaternion by specifying the 3x3 rotation matrix 
    (as a numpy array) from which the quaternion's rotation should be created.
    """
    try:
        shape = matrix.shape
    except AttributeError:
        raise TypeError("Invalid matrix type: Input must be a 3x3 numpy matrix")

    if shape == (3, 3):
        R = matrix
    else:
        raise ValueError("Invalid matrix shape: Input must be a 3x3 or 4x4 numpy array or matrix")

    # Check matrix properties
    if not np.allclose(np.dot(R, R.conj().transpose()), np.eye(3), atol=1e-5):
        raise ValueError("Matrix must be orthogonal, i.e. its transpose should be its inverse")
    if not np.isclose(np.linalg.det(R), 1.0):
        raise ValueError("Matrix must be special orthogonal i.e. its determinant must be +1.0")

    m = matrix # matrix.conj().transpose() # This method assumes row-vector and postmultiplication of that vector
    if m[2, 2] < 0:
        if m[0, 0] > m[1, 1]:
            t = 1 + m[0, 0] - m[1, 1] - m[2, 2]
            q = [m[1, 2]-m[2, 1],  t,  m[0, 1]+m[1, 0],  m[2, 0]+m[0, 2]]
        else:
            t = 1 - m[0, 0] + m[1, 1] - m[2, 2]
            q = [m[2, 0]-m[0, 2],  m[0, 1]+m[1, 0],  t,  m[1, 2]+m[2, 1]]
    else:
        if m[0, 0] < -m[1, 1]:
            t = 1 - m[0, 0] - m[1, 1] + m[2, 2]
            q = [m[0, 1]-m[1, 0],  m[2, 0]+m[0, 2],  m[1, 2]+m[2, 1],  t]
        else:
            t = 1 + m[0, 0] + m[1, 1] + m[2, 2]
            q = [t,  m[1, 2]-m[2, 1],  m[2, 0]-m[0, 2],  m[0, 1]-m[1, 0]]

    q = np.array(q)
    q *= 0.5 / sqrt(t);

    # Normalize again, there can be some numerical errors
    q = q / np.linalg.norm(q)
    return q

In [3]:
# def geodesic_distance(q1, q2):
#     from torch import dot, acos, abs
#     d = 2*acos(abs(torch.bmm(q1.view(q1.size()[0], 1, 4), q2.view(q2.size()[0], 4, 1))))
#     return d

from torch import dot, acos, abs
def geodesic_distance(q1, q2):
    val = torch.abs(torch.bmm(q1.view(q1.size()[0], 1, 4), q2.view(q2.size()[0], 4, 1)))
    for i, v in enumerate(val):
        if v > 1.0:
            val[i] = 1.0
    d = 2*torch.acos(val)
    return d

In [4]:
def geodesic_distance_num(q1, q2, th=1e-5):
    import numpy as np
    import math
    d = 2*math.acos(abs(np.dot(q1,q2)))
    if d < th:
        return 0.
    return d

In [12]:
from math import cos, sin, radians, degrees
import numpy as np
import torch

t = 45 

t = radians(t)

I = np.eye(3)

Rx = np.matrix([
    [1.,0.,0.],
    [0.,cos(t),-sin(t)],
    [0.,sin(t),cos(t)]
])
Ry = np.matrix([
    [cos(t),0.,sin(t)],
    [0.,1.,0.],
    [-sin(t),0.,cos(t)]
])

Real = np.matrix([
    [-0.0963063, 0.994044, -0.0510079 ],
    [-0.573321, -0.0135081, 0.81922 ],
    [0.813651, 0.10814, 0.571207 ]
])

q = [0,0,0,0,0]
q[0] = quaternion_from_matrix(Ry*Rx)
q[1] = quaternion_from_matrix(Real)
q[2] = quaternion_from_matrix(Rx)
q[3] = quaternion_from_matrix(Ry)
q[4] = quaternion_from_matrix(I)


print(q[1])

b1 = torch.tensor([q[1],q[2],q[3],q[0]])
# q1 = torch.unsqueeze(q1, 0)

b2 = torch.tensor([q[1],q[4],q[4],q[4]])
# q2 = torch.unsqueeze(q2, 0)
print("\n[")
for val in geodesic_distance(b1,b2):
    print("  " + str(degrees(val)))
print("]")

cor = geodesic_distance(b1, b2) < radians(10)
print(cor)

[0.60444039 0.29410654 0.35762815 0.64827098]

[
  0.0
  45.00000000000001
  45.00000000000001
  62.799429619838094
]
tensor([[[ 1]],

        [[ 0]],

        [[ 0]],

        [[ 0]]], dtype=torch.uint8)


In [11]:
real = np.matrix([
    [-0.113309, 0.991361,  -0.0660649 ],
    [-0.585432, -0.0128929, 0.810619 ],
    [0.802764,  0.130527, 0.581836]
])
 
print(real)

np.dot(real, real.conj().transpose())


[[-0.113309   0.991361  -0.0660649]
 [-0.585432  -0.0128929  0.810619 ]
 [ 0.802764   0.130527   0.581836 ]]


matrix([[ 1.00000013e+00, -2.66922000e-07,  5.40146000e-08],
        [-2.66922000e-07,  1.00000002e+00,  7.10877700e-07],
        [ 5.40146000e-08,  7.10877700e-07,  1.00000047e+00]])

In [140]:
abs(torch.bmm(b1.view(b1.size()[0], 1, 4), b2.view(b2.size()[0], 4, 1)))

tensor([[[ 0.4216]],

        [[ 1.0000]]], dtype=torch.float64)

In [133]:
np.dot(q1,q1)

1.0000002168414646

In [138]:
torch.acos(torch.tensor(1.000000))

tensor(0.)

In [139]:
import math
math.acos(1.000000)

0.0